In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
import os

# Step 1: Load and preprocess the dataset
df = pd.read_csv("/content/ProblemSolutionPythonV3.csv")
df = df.dropna(subset=["Python Code"])
df = df.rename(columns={"Problem": "Query", "Python Code": "Code_Snippet"})
df['Query'] = df['Query'].str.lower()
df['Code_Snippet'] = df['Code_Snippet'].str.lower()

# Step 2: Tokenize
tokenizer = T5Tokenizer.from_pretrained("t5-small")

def preprocess_data(data):
    inputs = ["generate code: " + query for query in data["Query"]]
    targets = data["Code_Snippet"].tolist()
    input_encodings = tokenizer(inputs, max_length=128, truncation=True, padding="max_length", return_tensors="pt")
    target_encodings = tokenizer(targets, max_length=128, truncation=True, padding="max_length", return_tensors="pt")
    return input_encodings, target_encodings

input_encodings, target_encodings = preprocess_data(df)

# Step 3: Custom Dataset
class CodeSnippetDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs["input_ids"])

    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs["input_ids"][idx],
            "attention_mask": self.inputs["attention_mask"][idx],
            "labels": self.targets["input_ids"][idx],
        }

dataset = CodeSnippetDataset(input_encodings, target_encodings)

# Step 4: Load Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)

# Step 5: Training arguments with improved config
output_dir = "/content/t5_finetuned150man_model"
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=150,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    save_total_limit=1,                    # Enable mixed precision if using GPU
)

# Step 6: Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
)

# Step 7: Train
trainer.train()

# Step 8: Save the model and tokenizer
os.makedirs(output_dir, exist_ok=True)
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
trainer.save_state()
torch.save(training_args, os.path.join(output_dir, "training_args.bin"))

# Step 9: Load back (optional)
model = T5ForConditionalGeneration.from_pretrained(output_dir).to(device)
tokenizer = T5Tokenizer.from_pretrained(output_dir)

# Step 10: Inference function
def generate_code(query):
    query = query.lower()
    input_text = "generate code: " + query
    input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True).to(model.device)
    outputs = model.generate(input_ids, max_length=512, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Step 11: Real-time Interface
while True:
    query = input("\n\nYour Query or type (exit) to leave : \n  ")
    if query.lower() == "exit":
        print("\nGood Bye...")
        break
    code = generate_code(query)
    print("\nCode Snippet: \n", code)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: allabonu-srisameera-ece23 (allabonu-srisameera-ece23-indian-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
500,3.481400
1000,2.627600
1500,2.408400
2000,2.252000
2500,2.135300
3000,2.033400
3500,2.006500
4000,1.905600
4500,1.854500
5000,1.822400




Your Query or type (exit) to leave : 
  exit

Good Bye...


In [2]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [5]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.3 MB/s eta 0:00:00


In [10]:
!pip install rouge_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=308b0ea0cb96aa8fd2d5841d7097314edb344ee1f66e700cb83d1f3b8dfd0503
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [13]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
import evaluate
from tqdm import tqdm
import pandas as pd

# ========== 1. Load Model and Tokenizer ==========
model_path = "/content/drive/MyDrive/t5_finetuned150man_model"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = T5ForConditionalGeneration.from_pretrained(model_path).to(device)
tokenizer = T5Tokenizer.from_pretrained(model_path)

# ========== 2. Load and Preprocess Dataset ==========
df = pd.read_csv("/ProblemSolutionPythonV3.csv")
df = df.dropna(subset=["Python Code"])
df = df.rename(columns={"Problem": "Query", "Python Code": "Code_Snippet"})
df['Query'] = df['Query'].str.lower()
df['Code_Snippet'] = df['Code_Snippet'].str.lower()

def preprocess_data(data):
    inputs = ["generate code: " + query for query in data["Query"]]
    targets = data["Code_Snippet"].tolist()
    input_encodings = tokenizer(inputs, max_length=128, truncation=True, padding="max_length", return_tensors="pt")
    target_encodings = tokenizer(targets, max_length=128, truncation=True, padding="max_length", return_tensors="pt")
    return input_encodings, target_encodings

input_encodings, target_encodings = preprocess_data(df)

# ========== 3. Create Custom Dataset ==========
class CodeSnippetDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs["input_ids"])

    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs["input_ids"][idx],
            "attention_mask": self.inputs["attention_mask"][idx],
            "labels": self.targets["input_ids"][idx],
        }

dataset = CodeSnippetDataset(input_encodings, target_encodings)

# ========== 4. Evaluation Function ==========
def evaluate_model(model, tokenizer, dataset, num_samples=100):
    bleu = evaluate.load("bleu")
    rouge = evaluate.load("rouge")
    exact_match_count = 0

    model.eval()
    inputs = dataset.inputs["input_ids"]
    targets = dataset.targets["input_ids"]
    total_samples = min(num_samples, len(inputs))

    for i in tqdm(range(total_samples)):
        input_ids = inputs[i].unsqueeze(0).to(model.device)
        labels = targets[i].unsqueeze(0).to(model.device)

        with torch.no_grad():
            outputs = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)

        pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
        true = tokenizer.decode(labels[0], skip_special_tokens=True)

        # BLEU evaluation: predictions should be a list of words (tokens)
        bleu.add(predictions=[pred.split()], references=[[true.split()]])

        # ROUGE evaluation: predictions and references should be plain strings
        rouge.add(prediction=pred, reference=true)

        if pred.strip() == true.strip():
            exact_match_count += 1

    bleu_score = bleu.compute()
    rouge_score = rouge.compute()
    exact_match = exact_match_count / total_samples

    print(f"\nEvaluation on {total_samples} samples:")
    print(f"BLEU Score: {bleu_score['bleu']:.4f}")
    print(f"ROUGE-L Score: {rouge_score['rougeL']:.4f}")
    print(f"Exact Match Accuracy: {exact_match * 100:.2f}%")

# ========== 5. Run Evaluation ==========
evaluate_model(model, tokenizer, dataset, num_samples=100)


  0%|          | 0/100 [00:04<?, ?it/s]


ValueError: Predictions and/or references don't match the expected format.
Expected format:
Feature option 0: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}
Feature option 1: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')},
Input predictions: [['import', 'numpy', 'as', 'np', 'x', '=', "np.array(['python", "exercises',", "'php',", "'c++'],", 'dtype=np.str)', 'print("original', 'array:")', 'print(x)', 'print("repeated', 'array', 'elements:")', 'print(x)']],
Input references: [[['import', 'numpy', 'as', 'np', 'x', '=', 'np.repeat(3,', '4)', 'print(x)', 'x', '=', 'np.array([[1,2],[3,4]])', 'print(np.repeat(x,', '2))']]]